In [1]:
import sys
sys.path.append('../')
import json
import pandas as pd
import sqlite3
from helper import load_dotenv
from tqdm.auto import tqdm

In [2]:
dotenv = load_dotenv()
con = sqlite3.connect( dotenv['DB_PATH'] )

In [3]:
data = pd.read_sql(
    '''
        SELECT * FROM adjustment_per_mask apm 
        WHERE apm.adjustment = 'trapezoidal_distortion'
    ''',
    con
)

# BackUp ziehen

In [ ]:
0 / 0

In [ ]:
data.to_pickle(dotenv['TEMP_PROCESSING_DIR'] / '24-04-28_trapezoidal_distortion_backup.pkl')

# Update

In [4]:
updates = []

for val in data.features:
    features = json.loads(val)
    if pd.isna(features['trapezoidal_distortion_strength_1']):
        features['trapezoidal_distortion_strength_1'] = 0

    if pd.isna(features['trapezoidal_distortion_strength_2']):
        features['trapezoidal_distortion_strength_2'] = 0
    
    if features['trapezoidal_distortion_direction'] == 'top':
        corner_adjustments = [
            (features['trapezoidal_distortion_strength_1'],0),
            (0,0),
            (0,0),
            (features['trapezoidal_distortion_strength_2'],0)
        ]
    elif features['trapezoidal_distortion_direction'] == 'left':
        corner_adjustments = [
            (features['trapezoidal_distortion_strength_2'],0),
            (features['trapezoidal_distortion_strength_1'],0),
            (0,0),
            (0,0)
        ]
    elif features['trapezoidal_distortion_direction'] == 'bottom':
        corner_adjustments = [
            (0,0),
            (features['trapezoidal_distortion_strength_2'],0),
            (features['trapezoidal_distortion_strength_1'],0),
            (0,0)
        ]
    else:
        corner_adjustments = [
            (0,0),
            (0,0),
            (features['trapezoidal_distortion_strength_1'],0),
            (features['trapezoidal_distortion_strength_2'],0)
        ]
    
    updates.append(json.dumps({ 'corner_adjustments' : corner_adjustments }))

In [5]:
len(updates), data.shape

(2318, (2318, 10))

In [6]:
data.iloc[10]

pdf_filename                                 004_2023_017_0_118_0.p1
job                                                           508001
type                                                              4c
variant_name                                          halftone600dpi
method                                                    soft_light
idx                                                                2
mask_id                                                   1EhfLLvuBf
adjustment                                    trapezoidal_distortion
features           {"trapezoidal_distortion_strength_1": 0.304858...
execution_index                                                    5
Name: 10, dtype: object

In [7]:
c = con.cursor()

for i in tqdm(range(len(updates))):
    row = data.iloc[i]
    
    c.execute(
        f'''
            UPDATE adjustment_per_mask
            SET features='{ updates[i] }'
            WHERE job='{ row.job }' AND pdf_filename='{ row.pdf_filename }' AND mask_id='{ row.mask_id }' AND adjustment='{ row.adjustment }'
        '''
    )

c.close()
con.commit()

  0%|          | 0/2318 [00:00<?, ?it/s]